In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%pip install netunicorn-base
%pip install netunicorn-client
%pip install netunicorn-library

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import time
import random
import time
import requests 
import re
import logging
import subprocess
from subprocess import Popen
from sys import platform
import os, sys
import logging
import json
import threading

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ping import Ping
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher
from netunicorn.base import Result, Failure, Success, Task, TaskDispatcher
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node

from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime

# Discussion Section Week 5
In this section we will practice running experiments with netUnicorn.

First we will get you setup with your netUnicorn API Credentials. If you submitted a project proposal, you should have received an email containing the credentials for accessing the netunicorn API. Please log on to the server and query the length and amount of available nodes in this deployment.

## NetUnicorn API Credentials

In [2]:
NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', '')       # substitute your login here
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', '') # substitue your password here

Display the available nodes in the netunicorn deployment. You should nodes of a few different types (aws, raspi, etc.)

In [4]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)
print(len(nodes))

Health Check: True
[<Uncountable node pool with next node template: [aws-fargate-A-, aws-fargate-B-, aws-fargate-ARM64-]>, [snl-server-5, atopnuc-84:47:09:1b:00:3c, atopnuc-84:47:09:16:b6:cf, atopnuc-84:47:09:17:c0:b6, atopnuc-84:47:09:17:c1:df, atopnuc-84:47:09:17:c1:8d, atopnuc-84:47:09:17:c8:0c, atopnuc-84:47:09:17:c4:83, atopnuc-84:47:09:17:c1:b7, atopnuc-84:47:09:17:c2:14, atopnuc-84:47:09:17:c0:57, atopnuc-84:47:09:17:c0:f6, raspi-e4:5f:01:56:d8:cd, raspi-e4:5f:01:56:d9:8b, raspi-e4:5f:01:a0:4f:96, raspi-e4:5f:01:56:d9:a3, raspi-e4:5f:01:9b:85:9c, raspi-e4:5f:01:a0:4a:dd, raspi-e4:5f:01:75:6e:53, raspi-e4:5f:01:79:4a:18, raspi-e4:5f:01:8d:f5:95, raspi-dc:a6:32:d7:6e:64, raspi-e4:5f:01:6f:ef:0a, raspi-e4:5f:01:8c:8a:41, raspi-e4:5f:01:ac:e5:4b, raspi-e4:5f:01:96:01:e8, raspi-e4:5f:01:9c:ca:3a, raspi-e4:5f:01:72:a3:1c, raspi-e4:5f:01:75:ae:8d, raspi-e4:5f:01:72:a2:35, raspi-e4:5f:01:ad:c9:2a, raspi-e4:5f:01:a7:ae:70, raspi-e4:5f:01:a0:34:a5, raspi-e4:5f:01:9c:20:81, raspi-e4:5f:01:

Last week we talked about the SpeedTest Task and how we can run speedtests on nodes and retrieve the results. Similarly, we have the Ping Task that we can use to ping an address. We can also specify the number of ping samples to generate.

In [3]:
pipeline = Pipeline().then(Ping(address='8.8.8.8', count=5))

In [4]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
nodes = client.get_nodes()
#working_nodes = ["raspi-e4:5f:01:56:d9:8b", "raspi-e4:5f:01:ad:c9:04"]
working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
experiment = Experiment().map(pipeline, working_nodes)
experiment_label = "test_ping"

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

# Prepare Experiment
client.prepare_experiment(experiment, experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

time.sleep(5)

# Execute Experiment
client.start_execution(experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED


In [5]:
# Get Results
from returns.pipeline import is_successful
from returns.result import Failure

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    if report.result is None:
        print("report.result is EMPTY..")
        continue

    result, log = report.result  # report stores results of execution and corresponding log

    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    if is_successful(result):
        data = result.unwrap()
    else:
        data = result.failure()
    try:
        for key, value in data.items():
            print(f"{key}: {value}")
    except:
        print(f"No attribute 'items' in result")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

Node name: raspi-e4:5f:01:56:d8:cd
Error: None
Result is: <class 'returns.result.Success'>
container_value: defaultdict(<class 'list'>, {'865dc04f-0205-4a67-9556-b1c006f2b1de': [<Success: {'container_value': PingResult(host='8.8.8.8', packets=[PacketResult(icmp_seq=0, ttl=58, time=4.39, unit='ms'), PacketResult(icmp_seq=1, ttl=58, time=4.129, unit='ms'), PacketResult(icmp_seq=2, ttl=58, time=4.37, unit='ms'), PacketResult(icmp_seq=3, ttl=58, time=4.276, unit='ms'), PacketResult(icmp_seq=4, ttl=58, time=6.009, unit='ms')], packet_loss=0.0, min_rtt=4.129, avg_rtt=4.635, max_rtt=6.009, stddev_rtt=0.693, unit_rtt='ms', unparsed_output=[], raw_output='PING 8.8.8.8 (8.8.8.8): 56 data bytes\n64 bytes from 8.8.8.8: icmp_seq=0 ttl=58 time=4.390 ms\n64 bytes from 8.8.8.8: icmp_seq=1 ttl=58 time=4.129 ms\n64 bytes from 8.8.8.8: icmp_seq=2 ttl=58 time=4.370 ms\n64 bytes from 8.8.8.8: icmp_seq=3 ttl=58 time=4.276 ms\n64 bytes from 8.8.8.8: icmp_seq=4 ttl=58 time=6.009 ms\n--- 8.8.8.8 ping statistic

You can observe that the raw_output, as well as the parsed output in this case, is returned in the results

## Traceroute Task
For this task you will be implementing a traceroute task, similar to the [ping](https://github.com/netunicorn/netunicorn-library/blob/main/tasks/measurements/ping.py) task.

In [6]:
import subprocess
from dataclasses import dataclass
from typing import List

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher

class TraceRoute(TaskDispatcher):
    def __init__(self, address: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.address = address
        self.linux_implementation = TraceRouteLinuxImplementation(self.address)

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation
        raise NotImplementedError(
            f'TraceRoute is not implemented for architecture: {node.architecture}'
        )

class TraceRouteLinuxImplementation(Task):
    
    requirements = ["apt-get install -y traceroute"]

    def __init__(self, address: str, *args, **kwargs):
        self.address = address.strip()
        super().__init__(*args, **kwargs)

    def run(self):
        result = subprocess.run(
            ["traceroute", self.address], capture_output=True
        )
        if result.returncode != 0:
            return Failure(
                result.stdout.decode("utf-8").strip()
                + "\n"
                + result.stderr.decode("utf-8").strip()
            )

        return self._format(result.stdout.decode("utf-8"))
    
    def _format(self, output):
        # Optional: Parse the output
        return output

In [7]:
# Define the pipeline to just run this new task
pipeline = Pipeline().then(TraceRoute(address='google.com'))

In [8]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
experiment = Experiment().map(pipeline, working_nodes)
experiment_label = "test_traceroute"

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

# Prepare Experiment
client.prepare_experiment(experiment, experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

time.sleep(5)

# Execute Experiment
client.start_execution(experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED


In [9]:
# Get Results
from returns.pipeline import is_successful
from returns.result import Failure

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    if report.result is None:
        print("report.result is EMPTY..")
        continue

    result, log = report.result  # report stores results of execution and corresponding log

    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    if is_successful(result):
        data = result.unwrap()
    else:
        data = result.failure()
    try:
        for key, value in data.items():
            print(f"{key}: {value}")
    except:
        print(f"No attribute 'items' in result")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

Node name: raspi-e4:5f:01:56:d8:cd
Error: None
Result is: <class 'returns.result.Success'>
container_value: defaultdict(<class 'list'>, {'22e2cabc-5dc9-4b7c-8b1d-10f7ac148813': [<Success: {'container_value': 'traceroute to google.com (142.250.68.78), 30 hops max, 60 byte packets\n 1  172.17.0.1 (172.17.0.1)  0.141 ms  0.049 ms  0.047 ms\n 2  10.1.1.1 (10.1.1.1)  0.749 ms  0.494 ms  0.436 ms\n 3  csworld52.cs.ucsb.edu (128.111.52.1)  1.952 ms  1.810 ms  1.671 ms\n 4  r1--535-c--1.commserv.ucsb.edu (128.111.252.148)  1.912 ms  1.705 ms  1.639 ms\n 5  dc--lax-agg10--ucsb--100g.cenic.net (137.164.23.90)  3.636 ms  3.934 ms  3.839 ms\n 6  72.14.222.56 (72.14.222.56)  3.685 ms  3.993 ms 74.125.49.165 (74.125.49.165)  5.548 ms\n 7  * 108.170.247.161 (108.170.247.161)  4.050 ms  4.005 ms\n 8  142.250.226.45 (142.250.226.45)  4.350 ms 142.250.226.43 (142.250.226.43)  4.709 ms  4.621 ms\n 9  142.250.226.43 (142.250.226.43)  4.919 ms lax31s11-in-f14.1e100.net (142.250.68.78)  3.633 ms  3.493 ms\n

## File Download Task
For this task you will be implementing a task to download a single file using `wget` and reporting the time it takes to finish downloading.

In [ ]:
import subprocess
from dataclasses import dataclass
from typing import List

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher


class FileDownload(TaskDispatcher):
    def __init__(self, address: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.address = address
        self.linux_implementation = FileDownloadLinuxImplementation(self.address)

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation
        raise NotImplementedError(
            f'TraceRoute is not implemented for architecture: {node.architecture}'
        )


class FileDownloadLinuxImplementation(Task):
    # Set the requirements to install the wget library
    requirements = ["apt-get install -y wget"]

    def __init__(self, address: str, *args, **kwargs):
        self.address = address.strip()
        super().__init__(*args, **kwargs)

    def run(self):
        # IMPLEMENT THIS FUNCTION

In [36]:
# Define the pipeline to just run this new task
pipeline = None

In [ ]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
experiment = Experiment().map(pipeline, working_nodes)
experiment_label = "test_traceroute"

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

# Prepare Experiment
client.prepare_experiment(experiment, experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

time.sleep(5)

# Execute Experiment
client.start_execution(experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

In [ ]:
# Get Results
from returns.pipeline import is_successful
from returns.result import Failure

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    if report.result is None:
        print("report.result is EMPTY..")
        continue

    result, log = report.result  # report stores results of execution and corresponding log

    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    if is_successful(result):
        data = result.unwrap()
    else:
        data = result.failure()
    try:
        for key, value in data.items():
            print(f"{key}: {value}")
    except:
        print(f"No attribute 'items' in result")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

## SpeedTest + File Download
Now we will attempt to run a speedtest alongside this file download to demonstrate how these active measurements can affect user traffic. Create a pipeline that starts a speedtest task using the StartSpeedTest task defined below, and then attempts to download a file. How different are the observed results?

In [ ]:
import subprocess
from typing import Dict

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher


class StartSpeedTest(TaskDispatcher):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.linux_instance = StartSpeedTestLinuxImplementation(name=self.name)

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_instance

        raise NotImplementedError(
            f'SpeedTest is not implemented for architecture: {node.architecture}'
        )


class StartSpeedTestLinuxImplementation(Task):
    requirements = ["pip install speedtest-cli"]

    def run(self):
        proc = subprocess.Popen(["speedtest-cli", "--simple", "--secure"], capture_output=False)
        time.sleep(2)
        if (exit_code := proc.poll()) is None:  # not finished yet
            return Success(proc.pid)
        return Failure(f"Speedtest terminated with return code {exit_code}")

class StopSpeedTestLinuxImplementation(Task):
    kill('speedtest-cli')

In [ ]:
# Define the pipeline to just run this experiment
pipeline = Pipeline().then(StartSpeedTest()).then(FileDownload())

In [ ]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
nodes = client.get_nodes()
working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
experiment = Experiment().map(pipeline, working_nodes)
experiment_label = "test_traceroute"

try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

# Prepare Experiment
client.prepare_experiment(experiment, experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

time.sleep(5)

# Execute Experiment
client.start_execution(experiment_label)
while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

In [ ]:
# Get Results
from returns.pipeline import is_successful
from returns.result import Failure

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    if report.result is None:
        print("report.result is EMPTY..")
        continue

    result, log = report.result  # report stores results of execution and corresponding log

    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    if is_successful(result):
        data = result.unwrap()
    else:
        data = result.failure()
    try:
        for key, value in data.items():
            print(f"{key}: {value}")
    except:
        print(f"No attribute 'items' in result")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

## SpeedTest + File Download
How do we need to change the implementation if we want to run a speedtest in the middle of a file download?